In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Vérification de l'installation de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    # On force la mise à jour vers la dernière version
    try:
        # git fetch récupère les infos, git reset --hard force l'état local à correspondre exactement au serveur
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour réussie (Dernier commit master).")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)
    print("✅ Installation terminée.")


In [ ]:
# === INSTALLATION CUSTOM NODES (Fix NumPy + Nodes + Modèles + Patch) ===

import os
import subprocess
import sys

COMFY_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFY_DIR}/custom_nodes"

print("🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...")

# --- 0. FIX CRITIQUE NUMPY & DÉPENDANCES SYSTÈME ---
print("🛠️ Préparation du système (Fix NumPy < 2.0)...")

# 1. Installer Aria2c
try:
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "aria2"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except:
    pass

# 2. FORCER LE DOWNGRADE DE NUMPY (Crucial pour éviter le crash InsightFace)
# On désinstalle et réinstalle une version compatible 1.x
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run([sys.executable, "-m", "pip", "install", "numpy<2"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("   ✅ NumPy corrigé (v1.x).")

# --- 1. LISTE DES NODES ---
nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",
    "https://github.com/rgthree/rgthree-comfy",
    "https://github.com/lldacing/ComfyUI_PuLID_Flux_ll",
    "https://github.com/cubiq/ComfyUI_essentials",
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",
    "https://github.com/Gourieff/ComfyUI-ReActor",
    "https://github.com/jags111/efficiency-nodes-comfyui",
    "https://github.com/city96/ComfyUI-GGUF",
    "https://github.com/ltdrdata/was-node-suite-comfyui",
    "https://github.com/1038lab/ComfyUI-QwenVL",
    "https://github.com/rzgarespo/ComfyUI-diffusiondb",
    "https://github.com/BizaNator/ComfyUI_PromptIterator",
    "https://github.com/ClownsharkBatwing/RES4LYF",
    "https://github.com/lquesada/ComfyUI-Inpaint-CropAndStitch",
    "https://github.com/ChangeTheConstants/SeedVarianceEnhancer",
    "https://github.com/PozzettiAndrea/ComfyUI-SAM3",
    
    
]

# --- 2. INSTALLATION INSIGHTFACE & NODES ---
print("📦 Installation de InsightFace & ONNX (peut prendre 1-2 min)...")
# On force la réinstallation pour être sûr qu'ils se lient au bon NumPy
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "insightface", "onnxruntime-gpu", "albumentations"], 
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except Exception as e:
    print(f"⚠️ Warning InsightFace: {e}")

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur clone {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} ok.")

# --- 3. TÉLÉCHARGEMENT MODÈLES ULTRALYTICS (YOLO) ---
print("\n👁️ Vérification des modèles de détection (YOLO)...")
base_yolo = f"{COMFY_DIR}/models/ultralytics"
bbox_path = os.path.join(base_yolo, "bbox")
segm_path = os.path.join(base_yolo, "segm")
os.makedirs(bbox_path, exist_ok=True)
os.makedirs(segm_path, exist_ok=True)

yolo_models = [
    # BBOX
    ("face_yolov8m.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt", bbox_path),
    ("face_yolov8n.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt", bbox_path),
    ("hand_yolov8s.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt", bbox_path),
    ("Eyes.pt", "https://huggingface.co/Tenofas/ComfyUI/resolve/d79945fb5c16e8aef8a1eb3ba1788d72152c6d96/ultralytics/bbox/Eyes.pt", bbox_path),
    # SEGM
    ("person_yolov8m-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt", segm_path),
    ("yolov8n-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/yolov8n-seg.pt", segm_path),
    ("face_yolov8m-seg_60.pt", "https://huggingface.co/bobmin/face_yolov8m-seg_60/resolve/main/face_yolov8m-seg_60.pt", segm_path)
]

for name, url, dest in yolo_models:
    full_path = os.path.join(dest, name)
    if not os.path.exists(full_path):
        print(f"   ⬇️ YOLO Download : {name}")
        # Méthode 1: Aria2c
        try:
            subprocess.run(["aria2c", "-x", "8", "-s", "8", "-k", "1M", "-d", dest, "-o", name, url], 
                           check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except:
            # Méthode 2: Wget (Fallback plus robuste)
            print(f"   ⚠️ Aria2c échoué, bascule sur wget pour {name}...")
            try:
                subprocess.run(["wget", "-q", "--show-progress", "-O", full_path, url], check=True)
            except Exception as e:
                print(f"   ❌ Échec total pour {name}. Vérifiez votre connexion.")

# --- 4. PATCH REACTOR NSFW ---
print("\n🔓 Application du patch ReActor (NSFW Bypass)...")
reactor_path = f"{CUSTOM_NODES_DIR}/ComfyUI-ReActor/scripts/reactor_sfw.py"
bypass_code = """
from PIL import Image
import logging
import os
from scripts.reactor_logger import logger
MODEL_EXISTS = True
def ensure_nsfw_model(nsfwdet_model_path): return True
def nsfw_image(img_data, model_path: str): return False
"""

if os.path.exists(os.path.dirname(reactor_path)):
    try:
        with open(reactor_path, "w") as f: f.write(bypass_code)
        print("   ✅ Patch appliqué.")
    except Exception as e:
        print(f"   ❌ Erreur patch : {e}")

print("\n🎉 INSTALLATION TERMINÉE ! Veuillez relancer ComfyUI.")

In [ ]:
# Z-IMAGE TURBO FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Z-Image Turbo) ---
db_models = {
    "✨ Nouveautés": [
        ("RealDream Z-Image V1 (FP8)", "https://civitai.com/api/download/models/2507717?type=Model&format=SafeTensor&size=pruned&fp=fp8", "/workspace/ComfyUI/models/diffusion_models", "realDream_zImageV1.safetensors", "civitai"),
    ],
    "🧬 Z-Image Turbo Core": [
        ("[11.7 GB] Z-Image Turbo BF16", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/diffusion_models/z_image_turbo_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_turbo_bf16.safetensors", "hf"),
        ("[12.6 GB] Z-Image Turbo BF16 NSFW V2", "https://huggingface.co/tewea/z_image_turbo_bf16_nsfw/resolve/main/z_image_turbo_bf16_nsfw_v2.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "z_image_turbo_bf16_nsfw_v2.safetensors", "hf"),
        ("[320 MB] VAE (ae.safetensors)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("[7.5 GB] Qwen 3.4B (Text Enc)", "https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors?download=true", "/workspace/ComfyUI/models/text_encoders", "qwen_3_4b.safetensors", "hf"),
    ],
    "🧪 LoRAs Personnels (ZIT Training)": [
        ("Grainscape Z-Image", "https://huggingface.co/Danrisi/GrainScape_UltraReal_ZImage/resolve/main/grainscape_zimage.safetensors", "/workspace/ComfyUI/models/loras", "grainscape_zimage.safetensors", "hf"),
        ("Tumblr Minimalism 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002250.safetensors", "hf"),
        ("Tumblr Minimalism 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/tmblr0ne_minimalism_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "tmblr0ne_minimalism_000002750.safetensors", "hf"),
        ("Hairfix 1500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001500.safetensors", "hf"),
        ("Hairfix 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000001750.safetensors", "hf"),
        ("Hairfix 2000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002000.safetensors", "hf"),
        ("Hairfix 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/v3sna-ZIT-hairfix_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "v3sna-ZIT-hairfix_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002250.safetensors", "hf"),
        ("Vy3sna Woman 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002500.safetensors", "hf"),
        ("Vy3sna Woman 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000002750.safetensors", "hf"),
        ("Vy3sna Woman 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003000.safetensors", "hf"),
        ("Vy3sna Woman 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/vy3snawoman-zit_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "vy3snawoman-zit_000003250.safetensors", "hf"),
        ("Vyesna ZIT v1 1750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000001750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000001750.safetensors", "hf"),
        ("Vyesna ZIT v1 2250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002250.safetensors", "hf"),
        ("Vyesna ZIT v1 2500", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002500.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002500.safetensors", "hf"),
        ("Vyesna ZIT v1 2750", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000002750.safetensors", "hf"),
        ("Vyesna ZIT v1 3000", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003000.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003000.safetensors", "hf"),
        ("Vyesna ZIT v1 3250", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna-ZIT-lora_v1_000003250.safetensors?download=true", "/workspace/ComfyUI/models/loras", "Vyesna-ZIT-lora_v1_000003250.safetensors", "hf"),
    ],
    "🛠️ Upscale Models": [
        ("4x-UltraSharp", "https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "4x-UltraSharp.pth", "hf"),
        ("RealESRGAN x4plus", "https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "RealESRGAN_x4plus.pth", "hf"),
        ("Remacri Original", "https://huggingface.co/jaideepsingh/upscale_models/resolve/main/remacri_original.pth?download=true", "/workspace/ComfyUI/models/upscale_models", "remacri_original.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '250px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Z-Image)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Z-Image...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🧬 INSTALLATION Z-IMAGE TURBO (Et LoRAs Persos)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DÉMARRAGE COMFYUI (FIX V7 : ULTIMATE FINAL - GIT DEPENDENCIES FIX)

import os
import subprocess
import sys

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation V7 - Dependance Git Fix)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/6] Vérification outils système...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2", "git", "cmake"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH
print("\n🔥 [2/6] Vérification Noyau PyTorch...")
try:
    import torch
    if "2.5.1" not in torch.__version__:
        raise ImportError
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
        "--index-url", "https://download.pytorch.org/whl/cu124"
    ])

# 3. RÉPARATION VERSIONS HUGGINGFACE
print("\n🤖 [3/6] Réparation HuggingFace & Transformers...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "transformers", "huggingface_hub>=0.25.0"])

# 4. INSTALLATION DÉPENDANCES PYPI (STANDARD)
print("\n📦 [4/6] Installation des librairies PyPI...")
pypi_packages = [
    # --- NOYAU ---
    "accelerate", "protobuf", "bitsandbytes", "ftfy", "google-generativeai",
    
    # --- BASES COMFYUI ---
    "numpy<2", "pydantic-settings", "sentencepiece", "aiohttp", "einops", "av", "torchsde", "scipy",
    "insightface", "onnxruntime-gpu", "kornia", "spandrel", "color-matcher",
    "alembic", "gguf", "segment-anything", "diffusers", "ultralytics", "piexif", "facexlib",
    "comfyui-workflow-templates", "flash-attn", "rotary_embedding_torch", "dill", "timm", "simpleeval", "flatten_json",

    # --- CUSTOM NODES DEPS ---
    "blend_modes", "facenet-pytorch", "mediapipe", "fvcore", "yapf", "omegaconf", "addict", 
    "trimesh[easy]", "pilgram", "rembg", "duckdb", "fairscale", "opencv-python", "matplotlib", "pycocotools", "pywavelets", "numpy>=1.26.4",
]
# On installe les standards d'abord
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pypi_packages)

# 4.1 INSTALLATION DÉPENDANCES GIT (WAS SUITE & EXOTIQUES)
print("\n🐙 [4.1/6] Installation des librairies Git (WAS Suite)...")
git_packages = [
    "git+https://github.com/WASasquatch/cstr.git",
    "git+https://github.com/WASasquatch/img2texture.git",
    "git+https://github.com/WASasquatch/ffmpy.git",
    "git+https://github.com/facebookresearch/sam2.git"
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + git_packages)

# 5. MODÈLES INSIGHTFACE
print("\n📥 [5/6] Vérification InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]
for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")
run_script = """#!/bin/bash
cd /workspace/ComfyUI
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet --enable-cors-header
"""
with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre...")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI (Mode Réparation V7 - Dependance Git Fix)...

🔧 [1/6] Vérification outils système...

🔥 [2/6] Vérification Noyau PyTorch...
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 22.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 187.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 127.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 264.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 210.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 28.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 44.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 40.3 MB/s eta 0:00:0000:0100:01
     ━━━━━


🤖 [3/6] Réparation HuggingFace & Transformers...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 835.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 107.1 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 150.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 271.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 kB 192.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 260.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 33.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📦 [4/6] Installation des librairies PyPI...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 89.9 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 17.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


...

  Resolved https://github.com/WASasquatch/ffmpy.git to commit f000737698b387ffaeab7cd871b0e9185811230d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/facebookresearch/sam2.git to /tmp/pip-req-build-mel56_vd


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2.git /tmp/pip-req-build-mel56_vd


  Resolved https://github.com/facebookresearch/sam2.git to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 2.3 MB/s eta 0:00:00 0:00:01
  Created wheel for cstr: filename=cstr-0.1.0-py3-none-any.whl size=2831 sha256=43cf07fea78dfcc16296b6d70c0b1c8ca9ccfb3e1d082a13c1edd56a9b0edc25
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1xb1his/wheels/8e/03/cf/9f370558fa4116c2f89e0df101413d76a2b954afd439737eea
  Created wheel for img2texture: filename=img2texture-1.0.6-py3-none-any.whl size=8356 sha256=c528d250020afc42838045b33ee4335497b452f6658700cb6dae3b4857418c2a
  Stored in directory: /tmp/pip-ephem-wheel-c


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📥 [5/6] Vérification InsightFace...

🚀 [6/6] Lancement du serveur...
✅ ComfyUI démarre...
[SAM3] ComfyUI-SAM3 prestartup script running...
[SAM3] Script directory: /workspace/ComfyUI/custom_nodes/comfyui-sam3
[SAM3] ComfyUI root: /workspace/ComfyUI
[SAM3] Copying image assets to /workspace/ComfyUI/input...
[SAM3] Image assets: 0 copied, 3 skipped
[SAM3] Prestartup script completed
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies. (GitPython)

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-01-13 09:12:00.344
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /usr/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/